In [1]:
# def
def add0(x, y=10):
    z = x+y
    return z
print(f"result:: {add0(10)}")

result:: 20


In [2]:
# 为加法函数添加计时功能
from time import time
def add1(x, y=10):
    start = time()
    z = x+y
    end = time()
    print(f"time: {end-start}")
    return z

result = add1(10)
print(f"end: {result}")

time: 0.0
end: 20


In [3]:
# 以函数作为参数
def timer0(func, x, y=10):
    start = time()
    res = func(x,y)
    end = time()
    print(f"time: {end-start}")
    return res

print(f"end : {timer0(add0, 10)}")
print(f"end : {timer0(add0, 10, 30)}")

time: 0.0
end : 20
time: 0.0
end : 40


timer(add0, 10, 30) 这种调用方法太复杂，可读性不好，这里有一种稍微简洁的做法。

In [4]:
# 返回函数
def timer1(func):
    def f(x,y=10):
        start = time()
        res = func(x,y)
        end = time()
        print(f"time: {end-start}")
        return res
    return f

In [5]:
add0 = timer1(add0)

@timer1
def add3(x, y=10):
    z = x+y
    return z

In [6]:
add0(10)

time: 0.0


20

In [7]:
add3(10)

time: 0.0


20

可以看到，timer1 以函数func作为参数，返回对func上进行包装后的函数f。对比timer0，timer1增加了函数的功能，保持了代码的可读性。上面的代码已经展示了装饰器的基本思想了，可是timer1中返回的函数f的参数必须和传入的函数func的参数对应，这有降低了函数的通用性。

# 装饰器

In [8]:
def timer(func):
    def f(*args,**kwargs):
        start = time()
        res = func(*args,**kwargs)
        end = time()
        print(f"time: {end-start}")
        return res
    return f

In [9]:
@timer
def sub():
    print("sub")
@timer
def mul(a, b):
    print(a*b)

In [10]:
sub()

sub
time: 0.0009996891021728516


In [11]:
mul(1,2)

2
time: 0.0


至此，我们了解了装饰器的基本使用方式

# 多装饰器

In [12]:
def test1(func):
    def wrapper(*args, **kwargs):
        print('before test1 ...')
        func(*args, **kwargs)
        print('after test1 ...')
    return wrapper #返回内层函数的引用

def test2(func):
    def wrapper(*args, **kwargs):
        print('before test2 ...')
        func(*args, **kwargs)
        print('after test2 ...')
    return wrapper #返回内层函数的引用

@test2
@test1
def add(a, b):
    print(a+b)

add(1, 2) #正常调用add

before test2 ...
before test1 ...
3
after test1 ...
after test2 ...


从上面的输出我们可以看到装饰器的执行顺序

![装饰器执行顺序](./img.png)

# 带参数的装饰器

我们知道装饰器最终返回的是嵌套函数的引用，只要记住这点，装饰器就任由我们发挥了。写一个带参数的装饰器：

In [13]:
def auth(permission):
    def _auth(func):
        def wrapper(*args, **kwargs):
            print(f"验证权限[{permission}]...")
            func(*args, **kwargs)
            print("执行完毕...")
        return wrapper
    return _auth


@auth("add")
def add(a, b):
    """
    求和运算
    """
    print(a + b)

add(1, 2)  # 正常调用add


验证权限[add]...
3
执行完毕...


In [14]:
print(add)
print(add.__name__)
print(add.__doc__)

<function auth.<locals>._auth.<locals>.wrapper at 0x00000141D1CD5F70>
wrapper
None


通过输出我们看出，add函数已经被改变。为了消除装饰器对原函数的影响，我们需要伪装成原函数，拥有原函数的属性，看起来就像是同一个人一样。functools.wraps为我们提供了便捷的方式

In [15]:
from functools import wraps

def auth(permission):
    def _auth(func):
        @wraps(func) # 注意此处
        def wrapper(*args, **kwargs):
            print(f"验证权限[{permission}]...")
            func(*args, **kwargs)
            print("执行完毕...")

        return wrapper

    return _auth


@auth("add")
def add(a, b):
    """
    求和运算
    """
    print(a + b)

print(add)
print(add.__name__)
print(add.__doc__)

<function add at 0x00000141D1CC1CA0>
add

    求和运算
    
